In [1]:
import os
import cv2
import random
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
from datetime import datetime
import matplotlib.pyplot as plt
from skimage.transform import resize
get_ipython().run_line_magic('matplotlib', 'inline') ###
################################################################################
import tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
################################################################################
from keras import layers,utils
from keras.datasets import cifar100
from keras.models import Sequential,load_model
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout, Activation, Flatten,Conv2D,MaxPooling2D, MaxPool2D,Add, concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D,LeakyReLU, Dense, InputLayer,Dropout, Activation, Flatten,Input, BatchNormalization, Conv2D, MaxPool2D, GlobalMaxPool2D,GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16, MobileNet, ResNet50, InceptionV3, Xception, VGG19,ResNet101
# from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
################################################################################
import gc
import tensorflow_probability as tfp
from tensorflow.keras import backend as K

In [2]:
# Global_Variables
inputs = Input(shape=(448, 448, 3))
sa_model_path = './models/MSE_MSLE_model001.h5'
pretrained_model = ResNet50(input_shape=(224, 224, 3),weights='imagenet', include_top=False)
final_model_name = 'Final_model'

In [3]:
# Some defined Loss Functions used by the Scale Adaptive model (Encoder model)
def rmse(y_true, y_pred):
  return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def custom_metric_SSIM_(y_true, y_pred):
        x=y_true
        y=y_pred
        mean_x=tf.math.reduce_mean(x)
        mean_y=tf.math.reduce_mean(y)

        var_x=tf.math.reduce_variance(x)
        var_y=tf.math.reduce_variance(y)
        covar_xy=tfp.stats.covariance(x,y)

        k1=0.01
        k2=0.03
        l=float(tf.math.pow(2,20) - 1)
        c1=float(k1 * l)
        c2=float(k2 * l)
        print(type(c1))
        c1 = tf.math.square(c1)
        c2 = tf.math.square(c2)
      # SSIM=(2μxμy + c1)(2σxy +c2)/((μx)2+(μy)2 +c1)((σx)2 +(σy)2 + c2)
        m=tf.math.multiply(2.0,covar_xy)
        t1=tf.math.add(m , c2)
        m=tf.math.multiply(mean_x,mean_y)
        m=float(m * 2.0)
        t2=tf.math.add(m ,c1)
        t3=(tf.math.square(mean_x)+tf.math.square(mean_y) +c1)
        t4=(tf.math.square(var_x) +tf.math.square(var_y) + c2)

        SSIM=tf.math.divide(tf.math.multiply(t2,t1),tf.math.multiply(t3,t4))
        return tf.math.reduce_mean(SSIM)

def custom_loss_MSEhigh_MSLElow_(y, y_pred):
        l1=tf.keras.losses.MeanSquaredError()
        l_high=l1(y[:,:,:,:12],y_pred[:,:,:,:12])

        l2=tf.keras.losses.MeanSquaredLogarithmicError()
        l_low=l2(y[:,:,:,12:],y_pred[:,:,:,12:])

        return 0.4 * l_high + 0.6 * l_low


In [4]:
# Function to load the encoder model or Scale Adaptive model
def load_encoder_model(model_path , inputs):
    try:
        resized_model = tf.keras.models.load_model(
            model_path,
            custom_objects={"custom_metric_SSIM_":custom_metric_SSIM_,
                            "custom_loss_MSEhigh_MSLElow_":custom_loss_MSEhigh_MSLElow_,
                            "rmse":rmse},
            compile=False)
        resized_model.trainable = False
        print(f"(✔) Model with name of '{model_path.split('/')[-1]}' is Successfuly Loaded....")
    except:
        print(f'(X) Cannot find the model in this path {model_path}')
        return None
    return resized_model(inputs) 

# Function to join the pretrained_model with the Scale Adaptive model for feature extraction
def build_feature_extractor_model(inputs , pretrained_model):
    # inputs here is the Scale Adaptive model
    try:
        feature_extractor_model = pretrained_model
        for layer in feature_extractor_model.layers:
            layer.trainable = True
        print(f"(✔) Successfully loaded the pretrained model for transfer learning and combine with the Scale Adaptive model.....")
    except:
        print(f'(X) Error while loading the pretrained model....')
        return None
    return feature_extractor_model(inputs)

# Function to attach the classification layers to classify our data
def attach_classifier(inputs):
    # inputs here is the feature extractor model which is (SA+ResNET50) so far
    try:
        X = GlobalAveragePooling2D()(inputs)
        X = Flatten()(X)

        X = Dense(units=512, activation='relu')(X)
        X = BatchNormalization()(X)
        X = Dropout(0.2)(X)

        X = Dense(units=256, activation='relu')(X)
        X = BatchNormalization()(X)
        X = Dropout(0.2)(X)

        X = Dense(units=128, activation='relu')(X)
        X = BatchNormalization()(X)
        X = Dropout(0.2)(X)

        X = Dense(units=4, activation='softmax', name=final_model_name)(X)
        print(f"(✔) Successfully attached the classification layers....")
    except:
        print(f'(X) Error while attaching the classification layers....')
        return None
    return X

In [5]:
resized_model = load_encoder_model(sa_model_path,inputs)

(✔) Model with name of 'MSE_MSLE_model001.h5' is Successfuly Loaded....


C:\Users\Mostafa\anaconda3\envs\capstone\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [6]:
resized_pretrained_model = build_feature_extractor_model(inputs= resized_model[:,:,:,12:] , pretrained_model= pretrained_model)

(✔) Successfully loaded the pretrained model for transfer learning and combine with the Scale Adaptive model.....


In [7]:
resized_pretrained_classification_model = attach_classifier(inputs= resized_pretrained_model)

(✔) Successfully attached the classification layers....


In [8]:
final_model = tf.keras.Model(inputs=inputs, outputs = resized_pretrained_classification_model)

In [9]:
final_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 448, 448, 3)]     0         
                                                                 
 req_dim_model (Functional)  (None, 224, 224, 15)      6214      
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 2048)              0     

In [10]:
final_model.name

'model'